**Why Data Reconciliation Matters?**

In a production environment, internal records and third-party payment gateways (like Stripe or PayPal) rarely match perfectly. Network timeouts, partial refunds, and duplicate API calls create 'data drift.' As an engineer, the job isn't just to move data, but to ensure its integrity.

To build a robust auditor, I need messy data. I’m using the Faker library to synthesize 100 transactions, then intentionally injecting 3 types of common financial errors: missing records, amount discrepancies, and duplicates. This ensures the logic is battle-tested against edge cases.

In [1]:
import pandas as pd
from faker import Faker
import random

fake = Faker()
Faker.seed(42)

In [2]:
rows = 100
data = [{
    "TXN_ID": f"TXN-{1000 + i}",
    "Customer": fake.name(),
    "Amount": round(random.uniform(50.0, 500.0), 2),
    "Date": fake.date_between(start_date='-30d', end_date='today')
} for i in range(rows)]

df_internal = pd.DataFrame(data)

# 2. Create Gateway Statement (The Messy Version)
# Start with a copy, then inject errors
df_gateway = df_internal.copy()

# A. Simulate 5 Missing Transactions (Drop them)
df_gateway = df_gateway.drop(random.sample(range(rows), 5))

# B. Simulate 3 Amount Mismatches (Pricing/Fee errors)
mismatch_indices = df_gateway.index[:3]
df_gateway.loc[mismatch_indices, 'Amount'] = df_gateway.loc[mismatch_indices, 'Amount'] - 1.00

# C. Simulate 2 Duplicates (Bank double-processing)
duplicates = df_gateway.iloc[-2:]
df_gateway = pd.concat([df_gateway, duplicates], ignore_index=True)

print(f"Internal Count: {len(df_internal)} | Gateway Count: {len(df_gateway)}")

Internal Count: 100 | Gateway Count: 97


In [3]:
# Step 1: Flag duplicates in the gateway before merging
df_gateway['is_duplicate'] = df_gateway.duplicated(subset=['TXN_ID'], keep=False)

# Step 2: Merge the two datasets
# We join on TXN_ID to see what's missing or different
comparison = pd.merge(
    df_internal, 
    df_gateway[['TXN_ID', 'Amount', 'is_duplicate']], 
    on="TXN_ID", 
    how="left", 
    suffixes=('_Internal', '_Gateway')
)

# Step 3: Categorize the Discrepancies
def identify_issue(row):
    if pd.isna(row['Amount_Gateway']):
        return "MISSING_IN_GATEWAY"
    elif row['is_duplicate'] == True:
        return "DUPLICATE_IN_GATEWAY"
    elif row['Amount_Internal'] != row['Amount_Gateway']:
        return "AMOUNT_MISMATCH"
    else:
        return "MATCHED"

comparison['Audit_Status'] = comparison.apply(identify_issue, axis=1)

# Display a summary of findings
print(comparison['Audit_Status'].value_counts())
comparison.head(10)

Audit_Status
MATCHED                 90
MISSING_IN_GATEWAY       5
DUPLICATE_IN_GATEWAY     4
AMOUNT_MISMATCH          3
Name: count, dtype: int64


,TXN_ID,Customer,Amount_Internal,Date,Amount_Gateway,is_duplicate,Audit_Status
0,TXN-1000,Allison Hill,496.60,2026-02-01,495.60,False,AMOUNT_MISMATCH
1,TXN-1001,Brian Yang,219.97,2026-02-17,218.97,False,AMOUNT_MISMATCH
2,TXN-1002,Javier Johnson,436.39,2026-02-01,435.39,False,AMOUNT_MISMATCH
3,TXN-1003,Lance Hoffman,97.74,2026-02-17,97.74,False,MATCHED
4,TXN-1004,Jerry Ramirez,378.44,2026-01-22,378.44,False,MATCHED
5,TXN-1005,Caitlin Henderson,395.46,2026-02-04,395.46,False,MATCHED
6,TXN-1006,Tyler Rogers,266.91,2026-01-26,266.91,False,MATCHED
7,TXN-1007,Ian Cooper,497.72,2026-02-03,497.72,False,MATCHED
8,TXN-1008,Monica Herrera,105.45,2026-02-09,105.45,False,MATCHED
9,TXN-1009,Edward Fuller,125.87,2026-02-08,125.87,False,MATCHED


In [4]:
def style_audit(row):
    color = ''
    if row.Audit_Status == "AMOUNT_MISMATCH":
        color = 'background-color: #ffcccc' # Red
    elif row.Audit_Status == "MISSING_IN_GATEWAY":
        color = 'background-color: #ffffcc' # Yellow
    elif row.Audit_Status == "DUPLICATE_IN_GATEWAY":
        color = 'background-color: #cce5ff' # Blue
    return [color] * len(row)

# Show the first 20 rows with styling applied
comparison.head(20).style.apply(style_audit, axis=1)

,TXN_ID,Customer,Amount_Internal,Date,Amount_Gateway,is_duplicate,Audit_Status
0,TXN-1000,Allison Hill,496.600000,2026-02-01,495.600000,False,AMOUNT_MISMATCH
1,TXN-1001,Brian Yang,219.970000,2026-02-17,218.970000,False,AMOUNT_MISMATCH
2,TXN-1002,Javier Johnson,436.390000,2026-02-01,435.390000,False,AMOUNT_MISMATCH
3,TXN-1003,Lance Hoffman,97.740000,2026-02-17,97.740000,False,MATCHED
4,TXN-1004,Jerry Ramirez,378.440000,2026-01-22,378.440000,False,MATCHED
5,TXN-1005,Caitlin Henderson,395.460000,2026-02-04,395.460000,False,MATCHED
6,TXN-1006,Tyler Rogers,266.910000,2026-01-26,266.910000,False,MATCHED
7,TXN-1007,Ian Cooper,497.720000,2026-02-03,497.720000,False,MATCHED
8,TXN-1008,Monica Herrera,105.450000,2026-02-09,105.450000,False,MATCHED
9,TXN-1009,Edward Fuller,125.870000,2026-02-08,125.870000,False,MATCHED


In [5]:
with pd.ExcelWriter("Reconciliation_Report.xlsx", engine='openpyxl') as writer:
    comparison.style.apply(style_audit, axis=1).to_excel(writer, index=False, sheet_name='Audit_Summary')

print("Report generated: Reconciliation_Report.xlsx")

Report generated: Reconciliation_Report.xlsx
